In [1]:
import pandas as pd

df = pd.read_csv("magnetism.csv")
df

,Timestamp,Elapsed time (minutes),Set B field (T),B field (T),Ramp speed (T/min),Voltage (V),Inner Lower Temperature,Outer Lower Temperature,Inner Upper Temperature,Outer Upper Temperature,1st Stage Temperature,Shield Temperature,2nd Stage Temperature,Magnetism_ID
0,02/08/2019 03:27:38 PM,2.805300,0.0,0.00000,0.000000,0.0,281.797983,285.092215,280.715422,283.969736,145.612162,279.920378,204.958080,./T10/20190208_SMS.txt
1,02/08/2019 03:28:01 PM,3.175539,0.0,0.00000,0.000000,0.0,281.768245,285.065814,280.686698,283.941646,145.521580,279.884569,204.900535,./T10/20190208_SMS.txt
2,02/08/2019 03:28:23 PM,3.554124,0.0,0.00000,0.000000,0.0,281.737322,285.041573,280.652349,283.915624,145.428632,279.849350,204.839601,./T10/20190208_SMS.txt
3,02/08/2019 03:28:46 PM,3.933961,0.0,0.00000,0.000000,0.0,281.704371,285.016645,280.621349,283.886878,145.337971,279.809776,204.778354,./T10/20190208_SMS.txt
4,02/08/2019 03:29:09 PM,4.314683,0.0,0.00000,0.000000,0.0,281.673613,284.987751,280.583256,283.856319,145.245554,279.771382,204.720981,./T10/20190208_SMS.txt
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2789982,03/20/2019 10:24:29 AM,9833.385706,0.0,-0.00000,0.006867,0.0,5.073398,5.019095,4.877788,4.965012,41.333217,63.687044,3.241164,././T10/20190313_SMS.txt
2789983,03/20/2019 10:24:35 AM,9833.491933,0.0,-0.00426,0.006867,0.0,5.071438,5.017297,4.876068,4.963015,41.332443,63.700309,3.242243,././T10/20190313_SMS.txt
2789984,03/20/2019 10:24:42 AM,9833.604840,0.0,-0.00000,0.006867,0.0,5.069822,5.015468,4.874303,4.960119,41.331908,63.710247,3.248909,././T10/20190313_SMS.txt
2789985,03/20/2019 10:24:48 AM,9833.707356,0.0,-0.00000,0.006867,0.0,5.068294,5.013746,4.872636,4.958283,41.331908,63.720780,3.325273,././T10/20190313_SMS.txt


In [2]:
float64_cols = list(df.select_dtypes(include='float64'))
df[float64_cols] = df[float64_cols].astype('float32')

In [3]:
df.iloc[:,3].values.dtype

dtype('float32')

In [4]:
import pandas as pd
from darts import TimeSeries

# Assuming 'your_dataset.csv' is a string containing the dataset

# df = pd.read_csv(pd.compat.StringIO('your_dataset.csv'))

# Convert the 'Timestamp' column to datetime type
df['Timestamp'] = pd.to_datetime(df['Timestamp'])

time_series_list = []
for ccr_id, group in df.groupby('Magnetism_ID'):
    # Sort the group by 'Timestamp' if not already sorted
    group.sort_values('Timestamp', inplace=True)
    
    # Set the 'Timestamp' column as the index
    group.set_index('Timestamp', inplace=True)
    
    # Exclude the 'CCR_ID' column from numeric conversion and resampling
    value_cols = ['Elapsed time (minutes)', 'Set B field (T)', 'B field (T)', 'Ramp speed (T/min)', 'Voltage (V)', 'Inner Lower Temperature', 'Outer Lower Temperature', 'Inner Upper Temperature', 'Outer Upper Temperature', '1st Stage Temperature', 'Shield Temperature', '2nd Stage Temperature', ]
    # value_cols = ['Elapsed time (minutes)','Setpoint (K)','Sensor A (K)','Sensor B (K)', 'Heater Output % (0-100)', 'Heater Range (integer 0-5)', 'Slope','Intercept','Sensor C (K) or sensor A (sensor unit)','Sensor D (K) or sensor B (sensor unit)']
    
    # Convert the numeric columns to numeric format, handling non-numeric values as NaN
    for col in value_cols:
        group[col] = pd.to_numeric(group[col], errors='coerce')
    
    # Select only numeric columns for resampling
    numeric_cols = group.select_dtypes(include='number').columns
    
    # Resample the numeric columns to a fixed frequency
    group_resampled = group[numeric_cols].resample('1T').mean()  # Adjust '5S' to the desired frequency
    
    # Handle missing values if necessary
    group_resampled = group_resampled.fillna(method='ffill')  # Forward fill missing values, or choose another appropriate method
    
    # Convert the resampled DataFrame to a Darts TimeSeries object
    time_series = TimeSeries.from_dataframe(group_resampled, value_cols=value_cols)
    
    time_series_list.append(time_series)


/tmp/ipykernel_77940/2093529370.py:9: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Timestamp'] = pd.to_datetime(df['Timestamp'])


In [5]:
# fix python path if working locally
import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt

from darts import TimeSeries
from darts.utils.timeseries_generation import (
    gaussian_timeseries,
    linear_timeseries,
    sine_timeseries,
)
from darts.models import (
    RNNModel,
    TCNModel,
    TransformerModel,
    NBEATSModel,
    BlockRNNModel,
    VARIMA,
)
from darts.metrics import mape, smape, mae
from darts.dataprocessing.transformers import Scaler
from darts.utils.timeseries_generation import datetime_attribute_timeseries
from darts.datasets import AirPassengersDataset, MonthlyMilkDataset, ElectricityDataset

import logging

logging.disable(logging.CRITICAL)

import warnings

warnings.filterwarnings("ignore")

%matplotlib inline

# for reproducibility
torch.manual_seed(1)
np.random.seed(1)

2023-07-28 14:52:33 darts.models WARNING: The statsforecast module could not be imported. To enable support for the StatsForecastAutoARIMA, StatsForecastAutoETS and Croston models, please consider installing it.


In [6]:
time_series_list_scaled = []

for time_series in time_series_list:
    scaler = Scaler()
    time_series_list_scaled.append(scaler.fit_transform(time_series))

In [7]:
# print(time_series_list_scaled[0].columns)
from darts import TimeSeries
temp = []
# Assuming you have a Darts TimeSeries object named 'ts'

# Convert the TimeSeries to a pandas DataFrame
for ts in time_series_list_scaled:
    df = ts.pd_dataframe()

    # Rename the columns in the pandas DataFrame
    new_column_names = ['Normalized Elapsed time (minutes)', 'Normalized Set B field (T)', 'Normalized B field (T)',
       'Normalized Ramp speed (T/min)', 'Normalized Voltage (V)', 'Normalized Inner Lower Temperature',
       'Normalized Outer Lower Temperature', 'Normalized Inner Upper Temperature',
       'Normalized Outer Upper Temperature', 'Normalized 1st Stage Temperature',
       'Normalized Shield Temperature', 'Normalized 2nd Stage Temperature']  # Replace with the new column names you want
    df.columns = new_column_names

    # Convert the modified pandas DataFrame back to a Darts TimeSeries
    temp.append(TimeSeries.from_dataframe(df))
time_series_list_scaled = temp
print(time_series_list_scaled[0].columns)

Index(['Normalized Elapsed time (minutes)', 'Normalized Set B field (T)',
       'Normalized B field (T)', 'Normalized Ramp speed (T/min)',
       'Normalized Voltage (V)', 'Normalized Inner Lower Temperature',
       'Normalized Outer Lower Temperature',
       'Normalized Inner Upper Temperature',
       'Normalized Outer Upper Temperature',
       'Normalized 1st Stage Temperature', 'Normalized Shield Temperature',
       'Normalized 2nd Stage Temperature'],
      dtype='object', name='component')


In [8]:
from darts import TimeSeries

covariates = []
for i in range(len(time_series_list)):
    ts_df = (time_series_list[i]).pd_dataframe()
    columns_to_keep = ['Set B field (T)', 'Elapsed time (minutes)', 'Ramp speed (T/min)', 'Inner Lower Temperature', 
                       'Outer Lower Temperature', 'Inner Upper Temperature',
                       'Outer Upper Temperature', '1st Stage Temperature',
                       'Shield Temperature', '2nd Stage Temperature']
    covariate_ts_df = ts_df[columns_to_keep]
    covariates.append(TimeSeries.from_dataframe(covariate_ts_df))


In [9]:
updated_covariates = []
for i in range(len(covariates)):
    df = covariates[i].pd_dataframe()
    # for time_series in time_series_list_scaled:
    temp = time_series_list_scaled[i].pd_dataframe()
    for col in temp.columns:
        df[col] = temp[col]
    updated_covariates.append(TimeSeries.from_dataframe(df))
    
print(updated_covariates[0].columns)
covariates = updated_covariates

Index(['Set B field (T)', 'Elapsed time (minutes)', 'Ramp speed (T/min)',
       'Inner Lower Temperature', 'Outer Lower Temperature',
       'Inner Upper Temperature', 'Outer Upper Temperature',
       '1st Stage Temperature', 'Shield Temperature', '2nd Stage Temperature',
       'Normalized Elapsed time (minutes)', 'Normalized Set B field (T)',
       'Normalized B field (T)', 'Normalized Ramp speed (T/min)',
       'Normalized Voltage (V)', 'Normalized Inner Lower Temperature',
       'Normalized Outer Lower Temperature',
       'Normalized Inner Upper Temperature',
       'Normalized Outer Upper Temperature',
       'Normalized 1st Stage Temperature', 'Normalized Shield Temperature',
       'Normalized 2nd Stage Temperature'],
      dtype='object', name='component')


In [10]:
from darts import TimeSeries

# ts_df = ts.pd_dataframe()
time_final = []
for i in range(len(time_series_list)):
    ts_df = (time_series_list[i]).pd_dataframe()
    columns_to_keep = ['B field (T)', 'Voltage (V)', ]
    covariate_ts_df = ts_df[columns_to_keep]
    time_final.append(TimeSeries.from_dataframe(covariate_ts_df))
    
print(len(time_final[0].columns))
# time_final = time_series_list

2


In [11]:
# load data
train, val = [], []
VAL_LEN = 200
temp1, temp2 = [], []

for i in range(len(time_final)):
    series = time_final[i]
    train.append(series[:-VAL_LEN])
    val.append(series[-VAL_LEN:])
    temp1.append(covariates[i][-VAL_LEN:])
    temp2.append(covariates[i][:-VAL_LEN])

# for time_series in time_final:
#     series = time_series.astype(np.float32)
#     train.append(series[:-VAL_LEN])
#     val.append(series[-VAL_LEN:])
    
    
# for time_series in covariates:
#     series = time_series.astype(np.float32)
#     temp.append(series[:-VAL_LEN])
val_past_covariates_past = temp1
covariates = temp2

# split in train / validation (note: in practice we would also need a test set)
# train, val = 


In [12]:
train[1].dtype

dtype('float32')

In [13]:
print(len(train[0]))
print(len(covariates[1]))

14390
9634


In [14]:
print(len(val[0]))

200


In [18]:
# train = train[:2]
# val = val[:2]
# covariates = covariates[:2]
# val_past_covariates_past = val_past_covariates_past[:2]

In [15]:
# class Trial():
#     def suggest_int(self, name, low, high):
#         return low
#     def suggest_categorical(self, name, cats):
#         return cats[0]
#     def suggest_float(self, name, low, high, log = False):
#         return low
        

In [20]:
import numpy as np
import optuna
import torch
from optuna.integration import PyTorchLightningPruningCallback
from pytorch_lightning.callbacks import EarlyStopping
from sklearn.preprocessing import MaxAbsScaler

from darts.dataprocessing.transformers import Scaler
from darts.datasets import AirPassengersDataset
from darts.metrics import smape
from darts.models import TCNModel
from darts.utils.likelihood_models import GaussianLikelihood

# scale
# scaler = Scaler(MaxAbsScaler())
# train = scaler.fit_transform(train)
# val = scaler.transform(val)

# define objective function
def objective(trial):
    # select input and output chunk lengths
    in_len = trial.suggest_int("in_len", 15, 120)
    out_len = trial.suggest_int("out_len", 1, in_len-1)

    # Other hyperparameters
    kernel_size = trial.suggest_int("kernel_size", 2, 5)
    num_filters = trial.suggest_int("num_filters", 1, 5)
    weight_norm = trial.suggest_categorical("weight_norm", [False, True])
    dilation_base = trial.suggest_int("dilation_base", 2, 4)
    dropout = trial.suggest_float("dropout", 0.0, 0.4)
    lr = trial.suggest_float("lr", 5e-5, 1e-3, log=True)
    # include_year = trial.suggest_categorical("year", [False, True])

    # throughout training we'll monitor the validation loss for both pruning and early stopping
    pruner = PyTorchLightningPruningCallback(trial, monitor="val_loss")
    early_stopper = EarlyStopping("val_loss", min_delta=0.001, patience=3, verbose=True)
    callbacks = [pruner, early_stopper]
    
    num_workers = 4
        
    pl_trainer_kwargs = {
        "accelerator": "gpu",
        "devices": [0],
        "callbacks": callbacks,
    }

    encoders = None #look into add encoders

    # reproducibility
    torch.manual_seed(42)
    # kernel_size', 'num_filters', 'weight_norm', 'dilation_base']`
    # build the TCN model
    model = TCNModel(
        input_chunk_length=in_len,
        output_chunk_length=out_len,
        batch_size=32,
        n_epochs=4,
        nr_epochs_val_period=1,
        kernel_size=kernel_size,
        num_filters=num_filters,
        weight_norm=weight_norm,
        dilation_base=dilation_base,
        dropout=dropout,
        optimizer_kwargs={"lr": lr},
        add_encoders=encoders,
        likelihood=GaussianLikelihood(),
        pl_trainer_kwargs=pl_trainer_kwargs,
        model_name="tcn_model_1",
        force_reset=True,
        save_checkpoints=True,
    )
    
    # model = TransformerModel(
    # input_chunk_length=in_len,
    # output_chunk_length=out_len,
    # model_name="test5",
    # activation="relu",
    # # random_state=42,
    # save_checkpoints=True,
    # # force_reset=True,
    # pl_trainer_kwargs = {"accelerator": "gpu", "devices": [1]}
    # )
    
    
    # when validating during training, we can use a slightly longer validation
    # set which also contains the first input_chunk_length time steps
    # model_val_set = scaler.transform(series[-(VAL_LEN + in_len) :])
    # model_val_set = []
    # for time_series in time_final:
    #     series = time_series.astype(np.float32)
    #     model_val_set.append(series[-(VAL_LEN + in_len) :])

    # train the model
    model.fit(
        series=train,
        val_series=val,
        num_loader_workers=1,
        past_covariates = covariates,
        val_past_covariates=val_past_covariates_past
    )
    
#     model.fit(
#         series=train,
#         past_covariates=covariates,
#     );

    # reload best model over course of training
    model = TCNModel.load_from_checkpoint("tcn_model")
    
    # Evaluate how good it is on the validation set, using sMAPE
    preds = model.predict(series=train[0], n=VAL_LEN)
    smapes = smape(val[0], preds, n_jobs=-1, verbose=True)
    smape_val = np.mean(smapes)
    # if smape_val== np.nan:
    #     print("nan")
    return smape_val if smape_val != np.nan else float("inf")


# for convenience, print some optimization trials information
def print_callback(study, trial):
    print(f"Current value: {trial.value}, Current params: {trial.params}")
    print(f"Best value: {study.best_value}, Best params: {study.best_trial.params}")


# optimize hyperparameters by minimizing the sMAPE on the validation set
def run_trial():
    study = optuna.create_study(direction="minimize")
    study.optimize(objective, n_trials=3, callbacks=[print_callback])
        
if __name__ == "__main__":
    run_trial()
    # objective(Trial())


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Predicting: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Predicting: 0it [00:00, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Current value: 200.0, Current params: {'in_len': 15, 'out_len': 7, 'kernel_size': 2, 'num_filters': 1, 'weight_norm': False, 'dilation_base': 3, 'dropout': 0.12395046909016019, 'lr': 0.0006677944890664167}
Best value: 200.0, Best params: {'in_len': 25, 'out_len': 11, 'kernel_size': 2, 'num_filters': 2, 'weight_norm': False, 'dilation_base': 2, 'dropout': 0.37849396036633787, 'lr': 0.00015865973725637096}
